**<H1>ЛЕКСИЧНИЙ АНАЛІЗ</H1>**

Автор: **Григорій ЖОЛТКЕВИЧ**

# **Підготовка блокноту до роботи**

In [43]:
from typing import Tuple, Sequence, Any
from typing_extensions import Self

# **Клас регулярних виразів**

In [44]:
class ReX(tuple):

    EMT = 0  # empty expression
    NIL = 1  # nil expression
    CHR = 2  # one letter expression
    AST = 3  # Kleene closure
    CAT = 4  # concatanation
    ALT = 5  # alternation

    def __new__(cls, kind: int, *args: Sequence[Any]) -> Self:
        if type(kind) != int:
            raise ValueError("ReX() error! Bad type of expression kind")
        if kind == ReX.EMT or kind == ReX.NIL:
            if args:
                raise ValueError("ReX() error! Unexpected argument(s)")
            return super().__new__(cls, (kind, ()))
        elif kind == ReX.CHR:
            if len(args) != 1:
                raise ValueError("ReX() error! Invalid number of arguments")
            if type(args[0]) != str and len(args[0]) != 1:
                raise ValueError("ReX() error! Bad argument(s)")
            return super().__new__(cls, (kind, args))
        elif kind == ReX.AST:
            if len(args) != 1:
                raise ValueError("ReX() error! Invalid number of arguments")
            if type(args[0]) != ReX:
                raise ValueError("ReX() error! Bad argument(s)")
            return super().__new__(cls, (kind, args))
        elif kind == ReX.CAT or kind == ReX.ALT:
            if len(args) != 2:
                raise ValueError("ReX() error! Invalid number of arguments")
            if not all(type(arg) == ReX for arg in args):
                raise ValueError("ReX() error! Bad argument(s)")
            return super().__new__(cls, (kind, args))
        else:
            raise ValueError("ReX() error! Bad kind of expression")

    @property
    def kind(self):
        return self[0]

    def __eq__(self, another: Self) -> bool:
        if type(another) != ReX:
            raise ValueError("invalid comparison")
        return super().__eq__(self, another)

    def __ne__(self, another):
        return not (self == another)

    def __str__(self):
        if self.kind == ReX.EMT:
            return "∅"
        if self.kind == ReX.NIL:
            return "ϵ"
        if self.kind == ReX.CHR:
            return self[1][0]
        if self.kind == ReX.AST:
            return f"{self[1][0]}*"
        if self.kind == ReX.CAT:
            return f"({self[1][0]} {self[1][1]})"
        # self.kind == ReX.ALT
        return f"({self[1][0]} | {self[1][1]})"

    @classmethod
    def Empty(cls) -> Self:
        return ReX(ReX.EMT)

    @classmethod
    def Nil(cls) -> Self:
        return ReX(ReX.NIL)

    @classmethod
    def Char(cls, ch: str) -> Self:
        return ReX(ReX.CHR, ch)

    @classmethod
    def Ast(cls, rex: Self) -> Self:
        return ReX(ReX.AST, rex)

    @classmethod
    def Cat(cls, rex1: Self, rex2: Self) -> Self:
        return ReX(ReX.CAT, rex1, rex2)

    @classmethod
    def Alt(cls, rex1: Self, rex2: Self) -> Self:
        return ReX(ReX.ALT, rex1, rex2)

In [45]:
empty = ReX.Empty()
print(f"empty = {empty}")
nil = ReX.Nil()
print(f"nil   = {nil}")
a, b = ReX.Char('a'), ReX.Char('b')
print(f"a  = '{a}', b = '{b}'")
ast = ReX.Ast(a)
print(f"ast   = {ast}")
cat = ReX.Cat(a, b)
print(f"cat   = {cat}")
alt = ReX.Alt(a, b)
print(f"alt   = {alt}")

empty = ∅
nil   = ϵ
a  = 'a', b = 'b'
ast   = a*
cat   = (a b)
alt   = (a | b)


# **Функція** `simplify()`

Функція `simplify(expr: ReX) -> ReX`
забезпечує рекурсивне спрощення регулярних виразів за правилами

- $(\emptyset\mid e)\longrightarrow e$
- $(e\mid\emptyset)\longrightarrow e$
- $(\emptyset\ e)\longrightarrow\emptyset$
- $(e\ \emptyset)\longrightarrow\emptyset$
- $(\epsilon\ e)\longrightarrow e$
- $(e\ \epsilon)\longrightarrow e$
- $(\emptyset)^\ast\longrightarrow\epsilon$
- $(\epsilon)^\ast\longrightarrow\epsilon$

In [46]:
def simplify(rex: ReX) -> ReX:
    # Simplify an alternation expression
    if rex.kind == ReX.ALT:
        left, right = rex[1]
        left, right = simplify(left), simplify(right)
        # If one side is empty, return the other
        if left.kind == ReX.EMT:
            return right
        if right.kind == ReX.EMT:
            return left
        return ReX.Alt(left, right)
    # Simplify a concatenation expression
    elif rex.kind == ReX.CAT:
        left, right = rex[1]
        left, right = simplify(left), simplify(right)
        # If either side is empty, return empty
        if left.kind == ReX.EMT or right.kind == ReX.EMT:
            return ReX.Empty()
        # If one side is nil, return the other
        if left.kind == ReX.NIL:
            return right
        if right.kind == ReX.NIL:
            return left
        return ReX.Cat(left, right)
    # Simplify a Kleene star expression
    elif rex.kind == ReX.AST:
        inner = simplify(rex[1][0])
        # Kleene star of empty or nil is nil
        if inner.kind == ReX.EMT or inner.kind == ReX.NIL:
            return ReX.Nil()
        return ReX.Ast(inner)
    # If the expression is already simple, return it as is
    return rex

# **Функція** `check_nil()`

Функція `does_contain_nil(rex: ReX) -> bool`
перевіряє чи належить порожнє слово регулярній мові, що визначається регулярним виразом `ReX`.

In [47]:
def does_contain_nil(rex: ReX) -> bool:
    # Check if the expression is nil
    if rex.kind == ReX.NIL:
        return True
    # Empty expressions do not contain nil
    elif rex.kind == ReX.EMT:
        return False
    # Single character expressions do not contain nil
    elif rex.kind == ReX.CHR:
        return False
    # Alternation contains nil if either operand contains nil
    elif rex.kind == ReX.ALT:
        return does_contain_nil(rex[1][0]) or does_contain_nil(rex[1][1])
    # Concatenation contains nil if both operands contain nil
    elif rex.kind == ReX.CAT:
        return does_contain_nil(rex[1][0]) and does_contain_nil(rex[1][1])
    # Kleene star always contains nil
    elif rex.kind == ReX.AST:
        return True
    # Default return False
    return False

# **Функція** `Brzozowski()`

Функція `Brzozowski(rex: ReX, ch: str) -> ReX`
обчислює похідну Бжозовськи за символом `ch` регулярного виразу `rex`.

In [48]:
def Brzozowski(rex: ReX, ch: str) -> ReX:
    # Derivative of an empty expression is empty
    if rex.kind == ReX.EMT:
        return ReX.Empty()
    # Derivative of a nil expression is empty
    elif rex.kind == ReX.NIL:
        return ReX.Empty()
    # Derivative of a character expression
    elif rex.kind == ReX.CHR:
        return ReX.Nil() if rex[1][0] == ch else ReX.Empty()
    # Derivative of an alternation is the alternation of derivatives
    elif rex.kind == ReX.ALT:
        return simplify(ReX.Alt(Brzozowski(rex[1][0], ch), Brzozowski(rex[1][1], ch)))
    # Derivative of a concatenation
    elif rex.kind == ReX.CAT:
        left, right = rex[1]
        # If left contains nil, take the derivative of both parts
        if does_contain_nil(left):
            return simplify(ReX.Alt(ReX.Cat(Brzozowski(left, ch), right), Brzozowski(right, ch)))
        else:
            return simplify(ReX.Cat(Brzozowski(left, ch), right))
    # Derivative of a Kleene star is the concatenation of the derivative of the inner expression and the star itself
    elif rex.kind == ReX.AST:
        inner = rex[1][0]
        return simplify(ReX.Cat(Brzozowski(inner, ch), ReX.Ast(inner)))
    # Default case returns empty
    return ReX.Empty()

In [49]:
if __name__ == "__main__":
    empty = ReX.Empty()
    nil = ReX.Nil()
    a, b, c, d = ReX.Char('a'), ReX.Char('b'), ReX.Char('c'), ReX.Char('d')

    print("Tests of simplify method:")
    expr1 = ReX.Alt(a, ReX.Empty())
    simplified_expr1 = simplify(expr1)
    print(f"simplify({expr1}) = {simplified_expr1}")

    expr2 = ReX.Cat(ReX.Nil(), b)
    simplified_expr2 = simplify(expr2)
    print(f"simplify({expr2}) = {simplified_expr2}")

    expr3 = ReX.Ast(ReX.Nil())
    simplified_expr3 = simplify(expr3)
    print(f"simplify({expr3}) = {simplified_expr3}")

    expr4 = ReX.Cat(ReX.Alt(a, b), ReX.Empty())
    simplified_expr4 = simplify(expr4)
    print(f"simplify({expr4}) = {simplified_expr4}")

    expr5 = ReX.Ast(ReX.Alt(a, ReX.Nil()))
    simplified_expr5 = simplify(expr5)
    print(f"simplify({expr5}) = {simplified_expr5}")

    expr6 = ReX.Cat(ReX.Nil(), ReX.Alt(a, b))
    simplified_expr6 = simplify(expr6)
    print(f"simplify({expr6}) = {simplified_expr6}\n")


    print("Tests of does_contain_nil method:")
    expr7 = ReX.Ast(b)
    contains_nil_expr7 = does_contain_nil(expr7)
    print(f"does_contain_nil({expr7}) = {contains_nil_expr7}")

    expr8 = ReX.Cat(c, d)
    contains_nil_expr8 = does_contain_nil(expr8)
    print(f"does_contain_nil({expr8}) = {contains_nil_expr8}")

    expr9 = ReX.Alt(c, ReX.Nil())
    contains_nil_expr9 = does_contain_nil(expr9)
    print(f"does_contain_nil({expr9}) = {contains_nil_expr9}")

    expr10 = ReX.Cat(ReX.Ast(a), ReX.Nil())
    contains_nil_expr10 = does_contain_nil(expr10)
    print(f"does_contain_nil({expr10}) = {contains_nil_expr10}")

    expr11 = ReX.Ast(ReX.Alt(a, b))
    contains_nil_expr11 = does_contain_nil(expr11)
    print(f"does_contain_nil({expr11}) = {contains_nil_expr11}\n")

    print("Tests of Brzozowski method:")
    expr12 = ReX.Alt(a, b)
    derivative_expr12 = Brzozowski(expr12, 'a')
    print(f"Brzozowski({expr12}, 'a') = {derivative_expr12}")

    expr13 = ReX.Cat(a, ReX.Ast(b))
    derivative_expr13 = Brzozowski(expr13, 'a')
    print(f"Brzozowski({expr13}, 'a') = {derivative_expr13}")

    expr14 = ReX.Alt(ReX.Cat(a, b), c)
    derivative_expr14 = Brzozowski(expr14, 'a')
    print(f"Brzozowski({expr14}, 'a') = {derivative_expr14}")

    expr15 = ReX.Ast(ReX.Cat(a, b))
    derivative_expr15 = Brzozowski(expr15, 'a')
    print(f"Brzozowski({expr15}, 'a') = {derivative_expr15}")

    expr16 = ReX.Cat(ReX.Alt(a, c), ReX.Ast(b))
    derivative_expr16 = Brzozowski(expr16, 'a')
    print(f"Brzozowski({expr16}, 'a') = {derivative_expr16}")

Tests of simplify method:
simplify((a | ∅)) = a
simplify((ϵ b)) = b
simplify(ϵ*) = ϵ
simplify(((a | b) ∅)) = ∅
simplify((a | ϵ)*) = (a | ϵ)*
simplify((ϵ (a | b))) = (a | b)

Tests of does_contain_nil method:
does_contain_nil(b*) = True
does_contain_nil((c d)) = False
does_contain_nil((c | ϵ)) = True
does_contain_nil((a* ϵ)) = True
does_contain_nil((a | b)*) = True

Tests of Brzozowski method:
Brzozowski((a | b), 'a') = ϵ
Brzozowski((a b*), 'a') = b*
Brzozowski(((a b) | c), 'a') = b
Brzozowski((a b)*, 'a') = (b (a b)*)
Brzozowski(((a | c) b*), 'a') = b*
